In [1]:
%load_ext autoreload
%autoreload 2
# from lib.polar_histogram import PolarHistogram
# from lib.histogram_grid import HistogramGrid
# from lib.path_planner import PathPlanner
from lib.robot import Robot

# HistogramGrid params
active_region_dimension = (30, 30)
resolution = 1 # 1cm per node
map_fname = 'map.txt'

# PolarHistogram params
num_bins = 12 # each bin is 36 degrees
target_location = (50, 50)

# Robot Params
init_location = (0, 0)
init_speed = 1

robot = Robot.from_map(map_fname, init_location, target_location, init_speed, active_region_dimension, resolution, num_bins)

# print(path_planner)
# print(robot.path_planner.get_best_angle())

get_best_angle: before smoothing
index, angle, certainty
0 0.0 0.0
1 30.0 0
2 60.0 0
3 90.0 0
4 120.0 0
5 150.0 0
6 180.0 1114.078085939495
7 210.0 0.0
8 240.0 0.0
9 270.0 955.0
10 300.0 0
11 330.0 0

get_best_angle: after smoothing
index, angle, certainty
0 0.0 173.63636363636363
1 30.0 86.81818181818181
2 60.0 101.27982599449955
3 90.0 202.5596519889991
4 120.0 303.8394779834986
5 150.0 491.93748579618
6 180.0 680.0354936088613
7 210.0 665.5738494325436
8 240.0 651.1122052562259
9 270.0 636.6505610799081
10 300.0 448.55255326722676
11 330.0 260.45454545454544



In [2]:
# from lib.histogram_grid import HistogramGrid

# map_fname = 'map.txt'
# resolution = 1 # node size = 1cm
# hg = HistogramGrid.build_histogram_from_txt(map_fname, resolution)
# print(*hg.histogram_grid, sep="\n")